In [1]:
import time
import pathlib 

import pandas as pd

from sklearn.model_selection import train_test_split

from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, pipeline
from datasets import Dataset, load_dataset

import tensorflow as tf

random_seed = 0

/home/valkea/Dev/OpenClassrooms/Projets_AI/P7/venvP7/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-10 13:46:44.473866: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 13:46:44.473908: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment" # 5 labels
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest" # 3 labels mais fait pour
model_name = "distilbert-base-uncased-finetuned-sst-2-english" # 2 labels
# finiteautomata/bertweet-base-sentiment-analysis ??
# "distilbert-base-uncased" --> fine-tuning

Un **transformer** est un modèle qui utilise l'attention pour augmenter la vitesse à laquelle il peut être entrainé.

# 1 - pipeline

In [3]:
sentiment_pipeline = pipeline("sentiment-analysis", top_k=10, model=model_name)

2022-08-10 13:46:48.532106: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-10 13:46:48.532136: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 13:46:48.532157: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (valkea-XPS): /proc/driver/nvidia/version does not exist
2022-08-10 13:46:48.532671: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers o

In [4]:
data = ["I really love it", "I really hate it"]

In [5]:
sentiment_pipeline(data)

[[{'label': 'POSITIVE', 'score': 0.9998788833618164},
  {'label': 'NEGATIVE', 'score': 0.00012104477355023846}],
 [{'label': 'NEGATIVE', 'score': 0.9996116757392883},
  {'label': 'POSITIVE', 'score': 0.0003883188182953745}]]

# 2 - AutoModel & AutoTokenizer

In [6]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
data = tokenizer(["I really hate it", "I really love it"], 
          truncation=True, padding=True, return_tensors="tf")

In [8]:
data

{'input_ids': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[ 101, 1045, 2428, 5223, 2009,  102],
       [ 101, 1045, 2428, 2293, 2009,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [9]:
tf_outputs = tf_model(data)
tf_outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 4.3365107, -3.516785 ],
       [-4.327714 ,  4.691517 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [10]:
tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
tf_predictions

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9.9961168e-01, 3.8831888e-04],
       [1.2104454e-04, 9.9987888e-01]], dtype=float32)>

# 3 - Dataset

### From CSV 1

In [11]:
dataset1 = load_dataset(
    'data',
    'csv',
    split='train',
    data_files={'data_nlp_1563108.csv'},
    column_names=['text', 'target'],
)

Using custom data configuration data-3d6d34e60550ee5b
Reusing dataset csv (/home/valkea/.cache/huggingface/datasets/csv/data-3d6d34e60550ee5b/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


In [12]:
dataset1

Dataset({
    features: ['text', 'target', '__index_level_0__', '__index_level_1__', '__index_level_2__', '__index_level_3__', '__index_level_4__', '__index_level_5__', '__index_level_6__'],
    num_rows: 1452792
})

In [13]:
dsplit1 = dataset1.train_test_split(test_size=0.1)
dsplit1

DatasetDict({
    train: Dataset({
        features: ['text', 'target', '__index_level_0__', '__index_level_1__', '__index_level_2__', '__index_level_3__', '__index_level_4__', '__index_level_5__', '__index_level_6__'],
        num_rows: 1307512
    })
    test: Dataset({
        features: ['text', 'target', '__index_level_0__', '__index_level_1__', '__index_level_2__', '__index_level_3__', '__index_level_4__', '__index_level_5__', '__index_level_6__'],
        num_rows: 145280
    })
})

In [14]:
dataset1[1]

{'text': 'bummer shoulda get day d',
 'target': '$ url$ - awww , that be a bummer . you shoulda get david carr of third day to do it . ; d',
 '__index_level_0__': '0',
 '__index_level_1__': "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D",
 '__index_level_2__': "$URL$ - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D",
 '__index_level_3__': "$ url$ - awww , that 's a bummer . you shoulda got david carr of third day to do it . ; d",
 '__index_level_4__': 'bummer shoulda got day d',
 '__index_level_5__': 'bummer shoulda got day d',
 '__index_level_6__': 'bummer shoulda get day d'}

In [15]:
dsplit1['test'][0]

{'text': 'get promotion favoooor',
 'target': 'just get a promotion ! favoooor !',
 '__index_level_0__': '1',
 '__index_level_1__': 'Just got a promotion!  favoooor!',
 '__index_level_2__': 'Just got a promotion! favoooor!',
 '__index_level_3__': 'just got a promotion ! favoooor !',
 '__index_level_4__': 'got promotion favoooor',
 '__index_level_5__': 'got promotion favoooor',
 '__index_level_6__': 'get promotion favoooor'}

### From CSV 2

In [16]:
dataset2 = Dataset.from_csv('data/data_nlp_1563108.csv')

Using custom data configuration default-c72cb7284115c351
Reusing dataset csv (/home/valkea/.cache/huggingface/datasets/csv/default-c72cb7284115c351/0.0.0)


In [17]:
dataset2

Dataset({
    features: ['target', 'text', 'text_clean', 'tokens', 'tokens_filtered_advanced', 'tokens_filtered_simple', 'lemmas_filtered_advanced', 'lemmas_filtered_simple', 'lemmas_not_filtered'],
    num_rows: 1452791
})

In [18]:
dsplit2 = dataset2.train_test_split(test_size=0.1)
dsplit2

DatasetDict({
    train: Dataset({
        features: ['target', 'text', 'text_clean', 'tokens', 'tokens_filtered_advanced', 'tokens_filtered_simple', 'lemmas_filtered_advanced', 'lemmas_filtered_simple', 'lemmas_not_filtered'],
        num_rows: 1307511
    })
    test: Dataset({
        features: ['target', 'text', 'text_clean', 'tokens', 'tokens_filtered_advanced', 'tokens_filtered_simple', 'lemmas_filtered_advanced', 'lemmas_filtered_simple', 'lemmas_not_filtered'],
        num_rows: 145280
    })
})

In [19]:
dataset2[1]

{'target': 0,
 'text': "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!",
 'text_clean': "is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!",
 'tokens': "is upset that he ca n't update his facebook by texting it ... and might cry as a result school today also . blah !",
 'tokens_filtered_advanced': 'upset update facebook texting cry result school today',
 'tokens_filtered_simple': 'upset update facebook texting cry result school today',
 'lemmas_filtered_advanced': 'upset update facebook texte cry result school today',
 'lemmas_filtered_simple': 'upset update facebook texte cry result school today',
 'lemmas_not_filtered': 'be upset that he can not update his facebook by texte it ... and might cry as a result school today also . blah !'}

In [20]:
dsplit2['test'][0]

{'target': 0,
 'text': "@theoopsgirl glad you had good time! Missed you as well, my silly manager going on holiday so I couldn't go! ",
 'text_clean': "glad you had good time! Missed you as well, my silly manager going on holiday so I couldn't go!",
 'tokens': "glad you had good time ! missed you as well , my silly manager going on holiday so i could n't go !",
 'tokens_filtered_advanced': 'glad good time missed silly manager going holiday',
 'tokens_filtered_simple': 'glad good time missed silly manager going holiday',
 'lemmas_filtered_advanced': 'glad good time miss silly manager go holiday',
 'lemmas_filtered_simple': 'glad good time miss silly manager go holiday',
 'lemmas_not_filtered': 'glad you have good time ! miss you as well , my silly manager go on holiday so I could not go !'}

### From Pandas dataframe

In [21]:
data_work = pd.read_csv(
    pathlib.Path(pathlib.Path().absolute(), 'data', 'data_nlp_1563108.csv'), 
    usecols=['target', 'text'],
    encoding='ISO-8859-1',
    #nrows=100000,
)
data_work.rename(columns={'text':'text', 'target':'label'}, inplace=True)
display(data_work.head(2), data_work.shape)

# Select samples
sample_size = 100000
data_work = data_work.groupby('label', group_keys=False).apply(lambda x: x.sample(sample_size//2, random_state=random_seed))
data_work.set_index('label', inplace=True, drop=True)
display(data_work.head(2), data_work.shape)

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...


(1452791, 2)

,text
label,
0,I dont have a background for my twitter stream...
0,I did many things but missed tweeting


(100000, 1)

In [22]:
dataset3 = Dataset.from_pandas(data_work)

In [23]:
dataset3

Dataset({
    features: ['text', 'label'],
    num_rows: 100000
})

In [24]:
dsplit3 = dataset3.train_test_split(test_size=0.1)
dsplit3

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 10000
    })
})

In [25]:
dataset3[1]

{'text': 'I did many things but missed tweeting ', 'label': 0}

In [26]:
dsplit3['test'][0]

{'text': 'Still awake, going to sleep soon I hope. Just getting some reading done  Not skyping though ',
 'label': 0}

---
---

In [27]:
dataset3[0]

{'text': 'I dont have a background for my twitter stream...does anyone ever check those things?? and if they do...what should mine be? help ',
 'label': 0}

In [28]:
dsplit3['test'][0]

{'text': 'Still awake, going to sleep soon I hope. Just getting some reading done  Not skyping though ',
 'label': 0}

In [29]:
dsplit3['test'][0]['text']

'Still awake, going to sleep soon I hope. Just getting some reading done  Not skyping though '

### Un test avec **quelques textes** 

#### avec le pipeline

In [30]:
test_txt = ["I hate this stuff", "I really love it"]

In [31]:
pp_outputs = sentiment_pipeline(test_txt)
pp_outputs

[[{'label': 'NEGATIVE', 'score': 0.999596893787384},
  {'label': 'POSITIVE', 'score': 0.0004031736170873046}],
 [{'label': 'POSITIVE', 'score': 0.9998788833618164},
  {'label': 'NEGATIVE', 'score': 0.00012104477355023846}]]

In [32]:
y_preds_proba = [[y['score'] for y in x if y['label'] == "POSITIVE"][0] for x in pp_outputs]
y_preds_proba

[0.0004031736170873046, 0.9998788833618164]

#### avec les Auto

In [33]:
test_txt = ["I hate this stuff", "I really love it"]
test_tokens = tokenizer(test_txt, truncation=True, padding="max_length", max_length=65, return_tensors="tf")

In [34]:
tf_outputs = tf_model(test_tokens)
tf_outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 4.3156376, -3.5001032],
       [-4.3277135,  4.691517 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [35]:
tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
tf_predictions

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9.9959689e-01, 4.0317344e-04],
       [1.2104466e-04, 9.9987888e-01]], dtype=float32)>

In [36]:
y_preds_proba = tf_predictions.numpy()
y_preds_proba = [x[1] for x in y_preds_proba]
y_preds_proba

[0.00040317344, 0.9998789]

### Un test avec **de nombreux textes**

#### avec le pipeline

In [37]:
test_txt = data_work.text[:100].to_list()
test_txt[:5]

['I dont have a background for my twitter stream...does anyone ever check those things?? and if they do...what should mine be? help ',
 'I did many things but missed tweeting ',
 '@JoAnneJoyM most probably - eww all that pasty white skin on show. ',
 'Ohh, how bad I sleep ',
 '@LornA_AlicE hell yeah u were texting me i remember, i was at my nans n u told me tragic news! boo hoo! n u went to amsterdam without me! ']

In [38]:
pp_outputs = sentiment_pipeline(test_txt)
pp_outputs[:5]

[[{'label': 'NEGATIVE', 'score': 0.9994959831237793},
  {'label': 'POSITIVE', 'score': 0.0005040332325734198}],
 [{'label': 'NEGATIVE', 'score': 0.9947373270988464},
  {'label': 'POSITIVE', 'score': 0.005262717138975859}],
 [{'label': 'NEGATIVE', 'score': 0.9963040351867676},
  {'label': 'POSITIVE', 'score': 0.003695984371006489}],
 [{'label': 'NEGATIVE', 'score': 0.9995403289794922},
  {'label': 'POSITIVE', 'score': 0.00045966703328303993}],
 [{'label': 'NEGATIVE', 'score': 0.9946225881576538},
  {'label': 'POSITIVE', 'score': 0.005377411376684904}]]

In [39]:
y_preds_proba = [[y['score'] for y in x if y['label'] == "POSITIVE"][0] for x in pp_outputs]
y_preds_proba[:5]

[0.0005040332325734198,
 0.005262717138975859,
 0.003695984371006489,
 0.00045966703328303993,
 0.005377411376684904]

#### avec les Auto

In [40]:
# OK
test_txt = data_work.text[:100].to_list()
test_tokens = tokenizer(test_txt, truncation=True, padding="max_length", max_length=65, return_tensors="tf")
test_tokens[:1]

[Encoding(num_tokens=65, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [41]:
test_tokens

{'input_ids': <tf.Tensor: shape=(100, 65), dtype=int32, numpy=
array([[  101,  1045,  2123, ...,     0,     0,     0],
       [  101,  1045,  2106, ...,     0,     0,     0],
       [  101,  1030, 23459, ...,     0,     0,     0],
       ...,
       [  101,  1045,  8415, ...,     0,     0,     0],
       [  101,  6295,  2035, ...,     0,     0,     0],
       [  101, 19752,  2664, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(100, 65), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [43]:
# OK
def preprocess_function(x):
    return tokenizer(x['text'], truncation=True, padding="max_length", max_length=65, return_tensors="tf")
test_tokens = preprocess_function(dsplit3['test'][:100])
test_tokens

{'input_ids': <tf.Tensor: shape=(100, 65), dtype=int32, numpy=
array([[  101,  2145,  8300, ...,     0,     0,     0],
       [  101,  1045,  2123, ...,     0,     0,     0],
       [  101,  1030,  8224, ...,     0,     0,     0],
       ...,
       [  101, 18168,  2290, ...,     0,     0,     0],
       [  101, 17006,  1010, ...,     0,     0,     0],
       [  101,  1030, 23787, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(100, 65), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
# 10000
t0 = time.perf_counter()
tf_outputs = tf_model(test_tokens)
#tf_outputs
print(f"Inference time: {(time.perf_counter() - t0):.2f}s")

In [44]:
# 100
t0 = time.perf_counter()
tf_outputs = tf_model(test_tokens)
#tf_outputs
print(f"Inference time: {(time.perf_counter() - t0):.2f}s")

Inference time: 3.64s


In [175]:
# 1000
t0 = time.perf_counter()
tf_outputs = tf_model(test_tokens)
#tf_outputs
print(f"Inference time: {(time.perf_counter() - t0):.2f}s")

Inference time: 45.24s


In [169]:
tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
tf_predictions[:5]

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[9.9949592e-01, 5.0403317e-04],
       [9.9473739e-01, 5.2627195e-03],
       [9.9630404e-01, 3.6959790e-03],
       [9.9954027e-01, 4.5966770e-04],
       [9.9462253e-01, 5.3774193e-03]], dtype=float32)>

In [170]:
y_preds_proba = tf_predictions.numpy()
y_preds_proba = [x[1] for x in y_preds_proba]
y_preds_proba[:5]

[0.0005040332, 0.0052627195, 0.003695979, 0.0004596677, 0.0053774193]

---
---

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)
 
tokenized_test = dataset_df_split['train'].map(preprocess_function, batched=True)
tokenized_test

In [ ]:
tokenized_test[0]

In [ ]:
tf_outputs = tf_model(tokenized_test)
tf_outputs

In [ ]:
def encode(x):
    #return tokenizer(str(x), truncation=True, padding="max_length", return_tensors="tf")
    return tokenizer(str(x), truncation=True, padding=True, return_tensors="tf")

In [ ]:
data_test = dataset_df_split['test'].map(encode , batched=True)
data_test

In [ ]:
# Create new index
train_idx = [i for i in range(len(X_train.index))]
test_idx = [i for i in range(len(X_test.index))]
val_idx = [i for i in range(len(X_valid.index))]

# Convert to numpy
x_train = X_train.values[train_idx]
x_test = X_test.values[test_idx]
x_val = X_valid.values[val_idx]

#y_train = y_train.values[train_idx]
#y_test = y_test.values[test_idx]
#y_val = y_valid.values[val_idx]

In [ ]:
# Tokenize datasets
tr_tok = tokenizer(list(x_train), return_tensors='tf', truncation=True, padding=True, max_length=128)
val_tok = tokenizer(list(x_val), return_tensors='tf', truncation=True, padding=True, max_length=128)
test_tok = tokenizer(list(x_test), return_tensors='tf', truncation=True, padding=True, max_length=128)

In [ ]:
y_preds_raw = model(test_tok)